In [ ]:
import numpy as np # linear algebra
import pathlib
from datetime import datetime, date
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, sum, count, col, format_number, monotonically_increasing_id, hour, mean,minute

Configuração do Spark

In [ ]:
import findspark
import os
os.environ["SPARK_HOME"] = "/Users/fabioleite/Downloads/spark-3.5.3-bin-hadoop3"
findspark.init()

In [ ]:
spark = SparkSession.builder.appName("Meu App") \
    .config("spark.executor.extraJavaOptions", "-Djava.security.manager=allow") \
    .getOrCreate()

### Leitura dos arquivos

In [ ]:
# Caminhos
path_accidents_in_the_Czech_Republic = "Road_accidents_in_the_Czech_Republic/arquivo_base/road_accidents_czechia_2016_2022.csv"
path_pedestrian = "Road_accidents_in_the_Czech_Republic/arquivo_base/pedestrian.csv"
# Leituras e tratamentos iniciais
df_pedestrians = spark.read.csv(path_pedestrian, header=True, sep=',')
df_accidents = spark.read.csv(path_accidents_in_the_Czech_Republic, header=True, sep=',')

In [ ]:
df_accidents.describe()
# df.printSchema()
# df.show(truncate=False)

### 1. Gere um arquivo chamado “alcohol_accidents.csv” a partir do arquivo “road_accidents_czechia_2016_2022.csv”, filtrar os acidentes em que houve detecção do ingerimento de bebida alcoólica. (filtrar pelo campo alcohol)

+ Gere um arquivo chamado “alcohol_accidents.csv”.

In [ ]:
# Agrupar por 'alcohol' e contar as ocorrências
df_alcohol_count = df_accidents.groupBy('alcohol').agg(count('*').alias('count'))

# Mostrar o resultado
df_alcohol_count.show(truncate=False)

In [ ]:
# Leitura e tratamento dos arquivos do aluno
path_alcohol_accidents_aluno = "Road_accidents_in_the_Czech_Republic/transformações/alcohol_accidents.csv"
df_alcohol_accidents_aluno = spark.read.csv(path_alcohol_accidents_aluno, header=True, sep=',')

In [ ]:
# prompt: preciso selecionar todas as ocorrências de registros do df_accidents que iniciam com o valor 'yes blood alcohol content' na coluna alcohol

# Selecionar todas as ocorrências de registros que iniciam com 'yes blood alcohol content' na coluna 'alcohol'
df_accidents_alcohol = df_accidents.filter(df_accidents.alcohol.startswith('yes blood alcohol content'))

# Mostrar o resultado
df_accidents_alcohol.show()

In [ ]:
df_accidents_ne_filtered = df_accidents.filter(df_accidents.alcohol.startswith('yes blood alcohol content'))
path_alcohol_accidents = "Road_accidents_in_the_Czech_Republic/transformações/alcohol_accidents.csv"
df_alcohol_accidents = spark.read.csv(path_alcohol_accidents, header=True, sep=',')


In [ ]:
df_alcohol_accidents.describe('alcohol').show()

In [ ]:
df_accidents_ne_filtered.describe('alcohol').show()

In [ ]:

# Comparar os dois DataFrames usando um inner join
df_comparison = df_alcohol_accidents.join(df_accidents_ne_filtered, on=['alcohol'], how='inner')

# Ou, se você quiser comparar apenas a coluna 'alcohol', pode usar algo como:
df_alcohol_accidents_alcohol_column = df_alcohol_accidents.select('alcohol')
df_accidents_ne_filtered_alcohol_column = df_accidents_ne_filtered.select('alcohol')
# Encontrar a diferença (valores únicos em cada DataFrame)
df_difference = df_alcohol_accidents_alcohol_column.subtract(df_accidents_ne_filtered_alcohol_column)
df_difference.show()

In [ ]:
# prompt: preciso exibir o total de registros do dataframe df_alcohol_accidents e df_accidents_ne_filtered

print("Total de registros em df_alcohol_accidents:", df_alcohol_accidents.count())
print("Total de registros em df_accidents_ne_filtered:", df_accidents_ne_filtered.count())

### 2. Gere um arquivo chamado ”accidents_NCBMV.csv”, filtre pelo campo “crash_kind” os acidentes em que não houve colisão entre veículos em movimento. (filtrando por: not an option It is not a collision between moving vehicles)

+ Gere um arquivo chamado “accidents_NCBMV.csv”.

In [ ]:
df_accidents_filtered_crash_kind = df_accidents.filter(df_accidents.crash_kind == 'not an option It is not a collision between moving vehicles')
df_accidents_filtered_crash_kind.show()

In [ ]:
df_accidents_filtered_crash_kind.describe('crash_kind').show()

In [ ]:
path_accidents_NCBMV = 'Road_accidents_in_the_Czech_Republic/transformações/accidents_NCBMV.csv'
df_accidents_NCBMV = spark.read.csv(path_accidents_NCBMV, header=True, sep=',')

In [ ]:
df_accidents_NCBMV.describe('crash_kind').show()

In [ ]:
# Comparar os dois DataFrames usando um inner join
df_comparison = df_accidents_filtered_crash_kind.join(df_accidents_NCBMV, on=['crash_kind'], how='inner')

# Ou, se você quiser comparar apenas a coluna 'alcohol', pode usar algo como:
df_accidents_filtered_crash_kind_column = df_accidents_filtered_crash_kind.select('crash_kind')
df_accidents_NCBMV_column = df_accidents_NCBMV.select('crash_kind')
# Encontrar a diferença (valores únicos em cada DataFrame)
df_difference = df_accidents_filtered_crash_kind_column.subtract(df_accidents_NCBMV_column)
df_difference.show()

In [ ]:
print("Total de registros em df_accidents_filtered_crash_kind:", df_accidents_filtered_crash_kind.count())
print("Total de registros em df_accidents_NCBMV:", df_accidents_NCBMV.count())

### 3. Considere o arquivo gerado no item 2 (accidents_NCBMV.csv), filtre pelo campo “accident_kind” os acidentes causados por animais da floresta. (collision with forest animals)
+ Gere um arquivo chamado “accidents_CWFA.csv”.

In [ ]:
path_accidents_NCBMV_animals = 'Road_accidents_in_the_Czech_Republic/transformações/accidents_NCBMV.csv'
df_accidents_NCBMV = spark.read.csv(path_accidents_NCBMV, header=True, sep=',')

In [ ]:
df_accidents_animals = df_accidents.filter(df_accidents.accident_kind == 'collision with forest animals')
df_accidents_animals.describe('accident_kind').show()

In [ ]:
df_accidents_NCBMV.describe('accident_kind').show()

In [ ]:
df_comparison = df_accidents_animals.join(df_accidents_NCBMV, on=['accident_kind'], how='inner')

# Ou, se você quiser comparar apenas a coluna 'alcohol', pode usar algo como:
df_accidents_animals_column = df_accidents_animals.select('accident_kind')
df_accidents_NCBMV_column = df_accidents_NCBMV.select('accident_kind')
# Encontrar a diferença (valores únicos em cada DataFrame)
df_difference = df_accidents_animals_column.subtract(df_accidents_NCBMV_column)
df_difference.show()

### 4. Filtre pela coluna “pedestrian_category.csv” no arquivo “pedestre.csv” os pedestres que estavam sob a influência de álcool.
+ Gere um arquivo chamado “drunk_pedestrians.csv”

In [ ]:
path_drunk_pedestrians = 'Road_accidents_in_the_Czech_Republic/transformações/drunk_pedestrians.csv'
df_accidents_drunk_pedestrians = spark.read.csv(path_drunk_pedestrians, header=True, sep=',')
# df_accidents_drunk_pedestrians.show(truncate=False)
df_drunk_pedestrians_grp= df_accidents_drunk_pedestrians.groupBy('pedestrian_category').agg(count('*').alias('count'))
df_drunk_pedestrians_grp.show(truncate=False)

In [ ]:
df_pedestrians.show(truncate=False)

In [ ]:
df_pedestrians.groupBy('pedestrian_condition').agg(count('*').alias('count')).show(truncate=False)

In [ ]:
# prompt: preciso selecionar os registros do df_pedestrians que possuem como valor para o campo pedestrian_condition começando com a string under the influence of alcohol

# Filtrar os pedestres que estavam sob a influência de álcool
df_drunk_pedestrians = df_pedestrians.filter(df_pedestrians.pedestrian_condition.startswith('under the influence of alcohol'))

# Mostrar o resultado
df_drunk_pedestrians.show()

# Salvar o resultado em um arquivo CSV (opcional)
# path_drunk_pedestrians = '/content/drive/MyDrive/UEPB_Drive/2024.2/LEDA/Accidents_Czech/transformações/drunk_pedestrians.csv'
# df_drunk_pedestrians.write.csv(path_drunk_pedestrians, header=True)

In [ ]:
# prompt: preciso comparar dois dataframes df_accidents_drunk_pedestrians e df_drunk_pedestrians mostrar as diferenças e imprimir o total de registros de cada um

# Comparar os dois DataFrames usando um inner join
df_comparison = df_accidents_drunk_pedestrians.join(df_drunk_pedestrians, on=['pedestrian_category'], how='inner')

# Ou, se você quiser comparar apenas a coluna 'pedestrian_category', pode usar algo como:
df_accidents_drunk_pedestrians_column = df_accidents_drunk_pedestrians.select('pedestrian_category')
df_drunk_pedestrians_column = df_drunk_pedestrians.select('pedestrian_category')

# Encontrar a diferença (valores únicos em cada DataFrame)
df_difference = df_accidents_drunk_pedestrians_column.subtract(df_drunk_pedestrians_column)
df_difference.show()

print("Total de registros em df_accidents_drunk_pedestrians:", df_accidents_drunk_pedestrians.count())
print("Total de registros em df_drunk_pedestrians:", df_drunk_pedestrians.count())

## Ordenações
+ Ordenar o arquivo pela data do acidente (campo date) em ordem decrescente, da data mais recente para a menos recente.
+  Ordenar o arquivo accidents_NCBMV.csv pela coluna “time” (campo time) em ordem crescente, do menor para o maior.
+ Ordenar o arquivo accidents_NCBMV.csv pelo tipo de comunicação (campo communication_kind), em ordem alfabética crescente.